In [1]:
from langchain_ollama import ChatOllama

from langchain.prompts import PromptTemplate

from langchain.chains import LLMChain

from langchain_core.messages import HumanMessage, SystemMessage

from langchain_core.output_parsers import StrOutputParser

from datasets import Dataset
from tqdm import tqdm


import pandas as pd

model = ChatOllama(
    model="gemma2:9b",
)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\importlib\__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet("hf://datasets/yunfan-y/trump-tweets-cleaned/data/train-00000-of-00001.parquet")

In [3]:
# ask a question
messages = [
    SystemMessage(content="Your are an AI model that generates questions for a dialogue with Donald Trump. You will be the person asking the questions and the given text is the answer by Donald Trump. The question answer pair will later be used to fine-tune an AI model so that this model impersonates Donald Trump. You only generate one question. You do not generate any other text besides the question."),
    HumanMessage(content="{text}"),
]


In [4]:
parser = StrOutputParser()

In [14]:
# convert to huggingface dataset
dataset = Dataset.from_pandas(df)
dataset.push_to_hub("yunfan-y/trump-qa")


Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yunfan-y/trump-qa/commit/1e5eaf3aafe8d83693170b4d0773596c642a2ca6', commit_message='Upload dataset', commit_description='', oid='1e5eaf3aafe8d83693170b4d0773596c642a2ca6', pr_url=None, pr_revision=None, pr_num=None)

In [6]:
# write a for loop to generate questions for each text in the dataframe
for index, row in tqdm(df.iterrows(), total=len(df)):
    messages[1].content = row['text']
    question = parser.invoke(model.invoke(messages))
    df.at[index, 'input'] = question


100%|██████████| 33596/33596 [3:25:30<00:00,  2.72it/s]  


In [8]:
# rename column text to output
df.rename(columns={'text': 'output'}, inplace=True)

